# HOMEWORK 1

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import pandas as pd
import numpy as np
import random as rnd

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

In [ ]:
plt.rcParams['figure.figsize'] = (5, 5)

# defining colormaps useful for the plotting
cmap_light = ListedColormap(['orange', 'cyan', 'cornflowerblue'])    
cmap_bold = ListedColormap(['darkorange', 'c', 'darkblue'])

# plotting of the decision boundaries
def plot_decision_boundaries(X, y, clf, h=.02):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02), np.arange(y_min, y_max, 0.02))

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    fig, ax = plt.subplots()
    ax.pcolormesh(xx, yy, Z, cmap=cmap_light)
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
                
    return ax
    
def plot_scatter(ax, x1, x2, y, title = "", xlabel = "", ylabel = ""):  
    scatter = ax.scatter(x1, x2, c = y, cmap = cmap_bold, edgecolor='k', s=50)
    ax.legend(handles=scatter.legend_elements()[0], labels = set(y))
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

# plotting the accuracy for different values of a specific parameter
def plot_accuracy_scores(values, acc_scores, parameter = "", ax = None, color = "blue", label = None):
    if ax == None: ax = plt.subplot()
    ax.plot([str(value) for value in values], acc_scores, c = color, marker = 'o', label = label)
    ax.set_xlabel(parameter)
    ax.set_ylabel("Accuracy")
    ax.set_title(f"Accuracy with respect to {parameter}")
    if(label != None): ax.legend()

## 0. Prepare the data

In [ ]:
# loading the Wine dataset from sklearn
wine = load_wine()

In [ ]:
# selecting only two attributes 
attr_idxs = [0,1]
attributes = np.array(wine.feature_names)[attr_idxs]
X = wine["data"][:,attr_idxs]
y = wine["target"]
labels = set(y)

fig = plt.figure()
scatter = plt.scatter(X[:, 0], X[:, 1], c = y, cmap = cmap_bold, s = 20)
plt.xlabel(attributes[0])
plt.ylabel(attributes[1])
plt.legend(handles = scatter.legend_elements()[0], labels = labels)
plt.title(f"2D representation of the dataset")

In [ ]:
# randomly splitting the data (5:2:3)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size = 0.3, shuffle = True, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, train_size = 5/7, shuffle = True, random_state = 42)

In [ ]:
# standardizing data
scaling_policy = None

if scaling_policy != None:
    
    if scaling_policy == "minmax":
        scaler = MinMaxScaler()
    else if scaling_policy == "standard"
        scaler = StandardScaler()
        
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_val = scaler.transform(X_val)

    scaler.fit(X_train_val)
    X_train_val = scaler.transform(X_train_val)
    X_test = scaler.transform(X_test)

## 1. K-Nearest-Neighbors

K-NN classification using different values of K.

In [ ]:
k_values = [1,2,5,7]
acc_scores = []

# trying with different values of K and plotting the decision boundaries
for K in k_values:
    clf = KNeighborsClassifier(n_neighbors = K)
    clf.fit(X_train, y_train)
    ax = plot_decision_boundaries(X_train, y_train, clf)
    plot_scatter(ax, X_train[:,0], X_train[:,1], y_train, title = f"K = {K}", xlabel = attributes[0], ylabel = attributes[1])
    
    y_pred = clf.predict(X_val)
    acc_scores.append(accuracy_score(y_val, y_pred))

In [ ]:
# comparing the different values of K through accuracy 
fig, ax = plt.subplots()
plot_accuracy_scores(k_values, acc_scores, ax = ax, parameter = "K")
# fig.savefig('images/accuracy_K')

In [ ]:
# evaluating the model on the test set using the most performing value of k on the validation set
best_k = k_values[np.argmax(acc_scores)]
clf = KNeighborsClassifier(n_neighbors = best_k)
clf.fit(X_train_val, y_train_val)

y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy on the test set  with K = {best_k} --> {acc:.2f}")

# see the mistakes made by the classifier
wrong_class_mask = y_test != y_pred
ax = plot_decision_boundaries(X_train, y_train, clf)
plot_scatter(ax, X_test[wrong_class_mask,0], X_test[wrong_class_mask,1], y_test[wrong_class_mask], title = f"Errors KNN", xlabel = attributes[0], ylabel = attributes[1])

## 2-3. Linear and RBF SVM

Comparing SVM classification using the **linear** and the **RBF** kernels with different values of C.

In [ ]:
# SVM classification using linear and rbf kernels

c_values = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

kernels = ['linear', 'rbf']
acc_scores = {kernel:[] for kernel in kernels}

for kernel in kernels:
    for C in c_values:
        clf = SVC(kernel = kernel, C = C)
        clf.fit(X_train, y_train)
        ax = plot_decision_boundaries(X_train, y_train, clf)
        plot_scatter(ax, X_train[:,0], X_train[:,1], y_train, title = f"C = {C}, kernel = {kernel.upper()}", xlabel = attributes[0], ylabel = attributes[1])
        y_pred = clf.predict(X_val)
        acc_scores[kernel].append(accuracy_score(y_val, y_pred))

In [ ]:
fig, ax = plt.subplots()
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

for i, kernel in enumerate(kernels): 
    best_c = c_values[np.argmax(acc_scores[kernel])]
    clf =  SVC(kernel = kernel, C = best_c)
    clf.fit(X_train_val, y_train_val)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"{kernel.upper()} : accuracy on the test set with C = {best_c} --> {acc:.2f}")
    
    color = colors[i]
    plot_accuracy_scores(c_values, acc_scores[kernel], parameter = "C", color = color, ax = ax, label = kernel)
    
    # see the mistakes made by the classifier
    wrong_class_mask = y_test != y_pred
    ax_scatter = plot_decision_boundaries(X_train_val, y_train_val, clf)
    plot_scatter(ax_scatter, X_test[wrong_class_mask,0], X_test[wrong_class_mask,1], y_test[wrong_class_mask], title = f"Errors {kernel} SVM", xlabel = attributes[0], ylabel = attributes[1])


## 4. GridSearch over C and gamma using the RBF kernel

In [ ]:
gamma_values = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 10000]
c_values = [0.1, 1, 10, 100, 1000]

scores = pd.DataFrame({'C': [], 'gamma': [], 'accuracy': []})

# gridsearch over C and gamma
for C in c_values:
    for gamma in gamma_values:
        clf = SVC(kernel = 'rbf', C = C, gamma = gamma)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_val)
        acc = accuracy_score(y_val, y_pred)
        scores = scores.append(pd.Series({'C': C, 'gamma': gamma, 'accuracy': acc}), ignore_index = True)

best_config = scores.iloc[scores['accuracy'].idxmax()]
print(f"Best configuration on validation set:\n{best_config}")
best_c = best_config['C']
best_gamma = best_config['gamma']

clf = SVC(kernel = 'rbf', C = best_c, gamma = best_gamma)
clf.fit(X_train_val, y_train_val) 
ax = plot_decision_boundaries(X_train_val, y_train_val, clf)
plot_scatter(ax, X_train_val[:,0], X_train_val[:,1], y_train_val, title = f"C = {best_c}, gamma = {best_gamma}", xlabel = attributes[0], ylabel = attributes[1])

y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\nAccuracy on test the test set with C = {best_c}, gamma = {best_gamma} --> {acc:.2f}")

wrong_class_mask = y_test != y_pred
ax = plot_decision_boundaries(X_train_val, y_train_val, clf)
plot_scatter(ax, X_test[wrong_class_mask,0], X_test[wrong_class_mask,1], y_test[wrong_class_mask], title = f"Errors gridsearch rbf SVM", xlabel = attributes[0], ylabel = attributes[1])


In [ ]:
# plotting an heatmap to better visualize the performances of the combinations of values
indexed_scores = pd.pivot_table(scores, values='accuracy', index = ['C'], columns = ['gamma'])
fig , ax = plt.subplots(figsize=(10,10))
sns.heatmap(indexed_scores, annot = True)

## 5. GridSearch with CV

In [ ]:
param_grid = {'C': c_values, 'gamma': gamma_values}
gridsearch = GridSearchCV(SVC(kernel = 'rbf'), param_grid = param_grid, scoring = 'accuracy', cv = 5)
gridsearch.fit(X_train_val, y_train_val)

clf = gridsearch.best_estimator_
clf.fit(X_train_val, y_train_val)

y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy on test the test set with C = {clf.get_params()['C']}, gamma = {clf.get_params()['gamma']} --> {acc:.2f}")

wrong_class_mask = y_test != y_pred
ax = plot_decision_boundaries(X_train_val, y_train_val, clf)
plot_scatter(ax, X_test[wrong_class_mask,0], X_test[wrong_class_mask,1], y_test[wrong_class_mask], title = f"Errors gridsearch-CV rbf SVM", xlabel = attributes[0], ylabel = attributes[1])